In [ ]:
# q2_stylegan_inversion_mse.py

import torch
from torchvision import transforms
from torchvision.utils import save_image
from tqdm import tqdm
from models.stylegan2 import Generator  # Update import as needed
from losses import cosine_loss, lpips_loss  # Assume LPIPS is wrapped

device = 'cuda'
latent_dim = 512
image_size = 1024
G = Generator(image_size).to(device).eval()  # Pretrained StyleGAN2
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor()
])

# Load 20 real images (faces)
real_images = load_images_from_folder("data/faces", num_images=20, transform=transform).to(device)

results = {'z': [], 'w': [], 'w+': []}
metrics = {'z': [], 'w': [], 'w+': []}

def invert_image(img, space='z'):
    if space == 'z':
        latent = torch.randn(1, latent_dim, requires_grad=True, device=device)
    elif space == 'w':
        latent = torch.randn(1, latent_dim, requires_grad=True, device=device)
    elif space == 'w+':
        latent = torch.randn(1, 18, latent_dim, requires_grad=True, device=device)

    optimizer = torch.optim.Adam([latent], lr=0.01)
    for _ in range(1000):
        optimizer.zero_grad()
        generated = G(latent, input_is_latent=True)
        loss = torch.nn.functional.mse_loss(generated, img.unsqueeze(0))
        loss.backward()
        optimizer.step()
    return latent.detach(), generated.detach()

for space in ['z', 'w', 'w+']:
    for i, img in enumerate(tqdm(real_images)):
        z_opt, gen = invert_image(img, space)
        diff = (img - gen.squeeze()).abs()
        lpips_val = lpips_loss(img.unsqueeze(0), gen).item()
        cosine = cosine_loss(img, gen).item()
        l2 = torch.nn.functional.mse_loss(img, gen.squeeze()).item()
        metrics[space].append((l2, lpips_val, cosine))
        save_image(gen, f"outputs/q2/{space}_{i}_gen.png")

# Calculate avg/std
for space in metrics:
    arr = torch.tensor(metrics[space])
    mean = arr.mean(dim=0)
    std = arr.std(dim=0)
    print(f"{space.upper()} - L2: {mean[0]:.4f}±{std[0]:.4f}, LPIPS: {mean[1]:.4f}±{std[1]:.4f}, Cosine: {mean[2]:.4f}±{std[2]:.4f}")


In [ ]:
# q2_stylegan_custom_loss.py

# Assume imports and model setup same as above

def custom_loss(x, y, alpha=1.0, beta=0.8, gamma=0.3):
    mse = torch.nn.functional.mse_loss(x, y)
    cos = cosine_loss(x, y)
    lp = lpips_loss(x, y)
    return alpha * mse + beta * lp + gamma * cos

# Repeat similar loop as in Script 1 but using `custom_loss(...)` instead


In [1]:
# q2_stylegan_crossover.py

# Load two images: content and style
z_content = invert_to_w_plus(img_content)
z_style = invert_to_w_plus(img_style)

# Split A: content (layers 1–9), style (10–18)
z_mix_a = torch.cat([z_content[:, :9], z_style[:, 9:]], dim=1)
# Split B: style (1–9), content (10–18)
z_mix_b = torch.cat([z_style[:, :9], z_content[:, 9:]], dim=1)

# Generate images
img_A = G(z_mix_a, input_is_latent=True)
img_B = G(z_mix_b, input_is_latent=True)

# Compute metrics between Ic/Is and IA/IB, etc.


NameError: name 'invert_to_w_plus' is not defined